In [1]:
import json

def generate_mermaid_from_workflow(json_data):
    nodes = json_data.get("nodes", [])
    
    # Cabeçalho do Mermaid
    mermaid_code = ["flowchart TD"]
    
    # Definições de Estilos (Classes CSS do Mermaid)
    styles = [
        "classDef default fill:#f9f9f9,stroke:#333,stroke-width:2px;",
        "classDef ifStyle fill:#ffd700,stroke:#333,stroke-width:2px,stroke-dasharray: 5 5;", # Amarelo tracejado
        "classDef apiStyle fill:#61dafb,stroke:#333,stroke-width:2px;", # Azul
        "classDef llmStyle fill:#ff9a9e,stroke:#333,stroke-width:2px;", # Rosa
        "classDef inputStyle fill:#90ee90,stroke:#333,stroke-width:2px;", # Verde claro
        "classDef endStyle fill:#333,stroke:#333,stroke-width:2px,color:#fff;" # Preto
    ]
    
    links = []
    class_assignments = []

    for node in nodes:
        n_id = node.get("id")
        n_type = node.get("type")
        
        # 1. Definição da Forma do Nó baseada no Tipo
        label = f"{n_id}<br/><small>{n_type}</small>"
        
        if n_type == "if-else":
            shape = f"{n_id}{{{label}}}" # Diamante
            class_assignments.append(f"class {n_id} ifStyle;")
            
            # Lógica específica para links de if-else
            action = node.get("action_config", {})
            true_node = action.get("true_node")
            false_node = action.get("false_node")
            
            if true_node:
                links.append(f"{n_id} -->|True| {true_node}")
            if false_node:
                links.append(f"{n_id} -->|False| {false_node}")

        elif n_type == "api":
            shape = f"{n_id}[[{label}]]" # Subrotina/Caixa
            class_assignments.append(f"class {n_id} apiStyle;")
            if "next" in node:
                links.append(f"{n_id} --> {node['next']}")

        elif n_type == "llm":
            shape = f"{n_id}(({label}))" # Círculo duplo ou diferente
            class_assignments.append(f"class {n_id} llmStyle;")
            if "next" in node:
                links.append(f"{n_id} --> {node['next']}")
                
        elif n_type == "input":
            shape = f"{n_id}[/{label}/]" # Paralelogramo
            class_assignments.append(f"class {n_id} inputStyle;")
            if "next" in node:
                links.append(f"{n_id} --> {node['next']}")
        
        elif n_id == "end_node" or "end" in n_id:
             shape = f"{n_id}((({n_id})))" # Círculo sólido
             class_assignments.append(f"class {n_id} endStyle;")
             
        else:
            shape = f"{n_id}({label})" # Padrão arredondado
            if "next" in node:
                links.append(f"{n_id} --> {node['next']}")
        
        mermaid_code.append(f"    {shape}")

    # Montagem final
    mermaid_code.append("")
    mermaid_code.extend([f"    {link}" for link in links])
    mermaid_code.append("")
    mermaid_code.extend([f"    {style}" for style in styles])
    mermaid_code.extend([f"    {cls}" for cls in class_assignments])

    return "\n".join(mermaid_code)

# Exemplo de uso:
# Read workflow JSON from file
with open("flow_definition.json", "r", encoding="utf-8") as f:
    workflow_json = json.load(f)

# Generate and print mermaid
print(generate_mermaid_from_workflow(workflow_json))

# print(generate_mermaid_from_workflow(workflow_json))

flowchart TD
    setup(setup<br/><small>fixed</small>)
    ask_user[/ask_user<br/><small>input</small>/]
    get_pokemon[[get_pokemon<br/><small>api</small>]]
    check_weight{check_weight<br/><small>if-else</small>}
    joke_node((joke_node<br/><small>llm</small>))
    simple_print(simple_print<br/><small>fixed</small>)
    end_node(((end_node)))

    setup --> ask_user
    ask_user --> get_pokemon
    get_pokemon --> check_weight
    check_weight -->|True| joke_node
    check_weight -->|False| simple_print
    joke_node --> end_node
    simple_print --> end_node

    classDef default fill:#f9f9f9,stroke:#333,stroke-width:2px;
    classDef ifStyle fill:#ffd700,stroke:#333,stroke-width:2px,stroke-dasharray: 5 5;
    classDef apiStyle fill:#61dafb,stroke:#333,stroke-width:2px;
    classDef llmStyle fill:#ff9a9e,stroke:#333,stroke-width:2px;
    classDef inputStyle fill:#90ee90,stroke:#333,stroke-width:2px;
    classDef endStyle fill:#333,stroke:#333,stroke-width:2px,color:#fff;
    clas